<a id='home'></a>
### Contents

[Imports](#Imports)

[Dataloading](#Dataloading)

<a id='Imports'></a>
### Imports
[back to contents](#home)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
from os.path import join as p_join
import sys
from typing import List, Tuple, Dict, Set, Any, Optional, Callable
from pathlib import Path
sys.path.insert(0, str(Path(os.getcwd()).parent))
import requests
import json
import pickle
from tqdm import tqdm
try:
    from fabulous import color as fb_color
    color_print = lambda x, color='green': print(getattr(fb_color, color)(x)) if 'fb_color' in globals() else print(x)
except Exception as e:
    color_print = lambda x, color='green': print(x)

from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
from src.common_utils import get_container_ipaddr

<a id='Dataloading'></a>
### Dataloading
[back to contents](#home)

In [6]:
MINIO_ACCESS_KEY = os.environ['MINIO_ACCESS_KEY']
MINIO_SECRET_KEY = os.environ['MINIO_SECRET_KEY']
POSTGRES_USER = os.environ['POSTGRES_USER']
POSTGRES_PASSWORD = os.environ['POSTGRES_PASSWORD']

In [7]:
import docker
docker_client = docker.from_env()
docker_client

minio_container = [container for container in docker_client.containers.list() 
 if 'minio' in str(container.name).lower()]
assert len(minio_container) == 1, 'There should be only one minio container'
minio_container = minio_container[0]
minio_container

MINIO_IPADDR = get_container_ipaddr(minio_container)
print(f"MINIO_IPADDR: {MINIO_IPADDR}")

MINIO_IPADDR: 172.27.0.2


In [19]:
from src.minio_utils import initialize_minio_client

In [27]:
import os
import sys
import typing as t

if 'PYTHONPATH' in os.environ:
	PROJECT_PATH = os.environ["PYTHONPATH"]
	sys.path.insert(0, PROJECT_PATH)
else:
	PROJECT_PATH = '..'

import docker
import minio
from minio import Minio

from src.common_utils import get_container_ipaddr  


def initialize_minio_client(
		ipaddr: t.Optional[str]=None,
		access_key: t.Optional[str]=None,
		secret_key: t.Optional[str]=None,
		port_number: int=9001,
) -> minio.api.Minio:
    """Returns initialized minio client"""
    if ipaddr is None:
        docker_client = docker.from_env()
        minio_container = [
            container for container in docker_client.containers.list() 
            if 'minio' in str(container.name).lower()
        ]
        assert len(minio_container) == 1, 'There should be only one minio container'
        minio_container = minio_container[0]
        ipaddr = get_container_ipaddr(container=minio_container)

    print(f"ipaddr: {ipaddr}")
    minio_access_key = (
	    access_key if access_key is not None else os.getenv('MINIO_ACCESS_KEY', "")
    )
    minio_secret_key = (
	    secret_key if secret_key is not None else os.getenv('MINIO_SECRET_KEY', "")
    )
    minio_client = Minio(
        endpoint=f"{ipaddr}:{port_number}",
        access_key=minio_access_key,
        secret_key=minio_secret_key,
        secure=False,
    )
    print(f"minio_client: {minio_client}")
    print(f'minio_client.list_buckets(): {minio_client.list_buckets()}')
    # checking client creation
    assert isinstance(minio_client.list_buckets(), list), \
	    'minio_client if not initialized!'
    
    return minio_client

In [28]:
minio_client = initialize_minio_client(
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY
)
minio_client

minio_client: <minio.api.Minio object at 0x7fb962aabf40>


InvalidResponseError: non-XML response from server; Response code: 400, Content-Type: text/xml; charset=utf-8, Body: <?xml version="1.0" encoding="UTF-8"?><Error><Code>InvalidArgument</Code><Message>S3 API Requests must be made to API port.</Message><RequestId>0</RequestId></Error>

In [26]:
for bucket in minio_client.list_buckets(): break

In [16]:
tmp_data = {'some_key': ['some', 'list', 'of', 'values']}
json.dump(
    obj=tmp_data, fp=open('/home/aiandrejcev/tmp.json', mode='w', encoding='utf-8'), 
    ensure_ascii=False, indent=2
)

In [18]:
minio_client.fput_object(
    bucket_name='ufc-raw-data',
    object_name='tmp_try.json',
    file_path='/home/aiandrejcev/tmp.json',
)

In [ ]:
minio_client = Minio(
    endpoint="172.17.0.2:9000",
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False,
)

minio_client.fget_object(
    bucket_name=args.minio_bucket_name,
    object_name=args.object_name,
    file_path=tmp_data_local_path,
)

all_fights_list = json.load(open(tmp_data_local_path, mode='r', encoding='utf-8'))
os.remove(tmp_data_local_path)

In [10]:
from sqlalchemy import create_engine

In [11]:
import docker
docker_client = docker.from_env()
docker_client

pg_container = [container for container in docker_client.containers.list() 
 if 'pgdatabase' in str(container.name).lower()]
assert len(pg_container) == 1, 'There should be only one postgres container!'
pg_container = pg_container[0]

POSTGRES_IPADDR = get_container_ipaddr(pg_container)
print(f"POSTGRES_IPADDR: {POSTGRES_IPADDR}")

POSTGRES_IPADDR: 172.27.0.4


In [12]:
eng = create_engine(
    f'postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_IPADDR}'
)
eng.connect()

In [13]:
with eng.connect() as conn:
    conn.execute('create schema if not exists raw_data')

In [26]:
df.to_sql(
    name='tmp_table',
    con=eng,
    schema='tmp',
    if_exists='replace',
    index=False,
)

15

In [28]:
pd.read_sql('select * from tmp.tmp_table', eng)

,var1,var2,var3,var4,var5
0,4,4,-3,-3,2
1,-3,4,2,-3,-5
2,-1,1,-1,-2,3
3,3,2,-2,-4,0
4,0,4,-1,-4,-4
5,4,-5,0,-4,0
6,1,3,-4,-1,-4
7,2,3,-5,0,-1
8,-4,4,-5,1,3
9,-2,0,-3,2,-3
